<a href="https://colab.research.google.com/github/tejeshbhalla/tensorflow_projects/blob/master/Emotion_Recognition_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import csv 
import shutil
!git clone https://github.com/muxspace/facial_expressions

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 38.73 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [ ]:
d={}  #key-emotion -- value - array storing path
with open('/content/facial_expressions/data/legend.csv','r') as file:
  reader=csv.reader(file)
  next(reader)

  for row in reader:
    if row[2].lower() in d:
      d[row[2].lower()].append(row[1])
    else:
      d[row[2].lower()]=[row[1]]  

In [ ]:
d.keys()

dict_keys(['anger', 'surprise', 'disgust', 'fear', 'neutral', 'happiness', 'sadness', 'contempt'])

In [ ]:
os.mkdir('data/train')
os.mkdir('data/test')

In [ ]:
for k in list(d.keys()):
  os.mkdir(f'data/train/{k}')
  os.mkdir(f'data/test/{k}')

In [ ]:
split_size=0.8
for label in d:
  images_arr=d[label]
  total_size=len(images_arr)
  split_val=int(0.8*total_size)
  images_train=images_arr[:split_val]
  images_test=images_arr[split_val:]

  for i in images_train:
    source=os.path.join('/content/facial_expressions/images',i)
    dest=os.path.join('/content/data/train',label,i)
    shutil.copyfile(source,dest)
  for i in images_test:
    source=os.path.join('/content/facial_expressions/images',i)
    dest=os.path.join('/content/data/test',label,i)
    shutil.copyfile(source,dest)


  


In [12]:
train_gen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,height_shift_range=0.1,width_shift_range=0.1)
train_generator=train_gen.flow_from_directory('/content/data/train',target_size=(100,100),class_mode='categorical')

test_gen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)
test_generator=train_gen.flow_from_directory('/content/data/test',target_size=(100,100),class_mode='categorical')

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Input,Flatten,GlobalMaxPool1D,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
model=Sequential()
model.add(Input(shape=(100,100,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D())


model.add(Conv2D(48,(3,3),activation='relu'))
model.add(MaxPool2D())


model.add(Conv2D(100,(3,3),activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(512,(3,3),activation='relu'))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(128,activation=tf.keras.layers.LeakyReLU()))
model.add(Dense(512,activation=tf.keras.layers.LeakyReLU()))
model.add(Dense(8,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=Adam(0.001),metrics=['accuracy'])



In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 48)        13872     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 100)       43300     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 100)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 512)         4

In [ ]:
r=model.fit_generator(train_generator,epochs=10,steps_per_epoch=10941//100,validation_data=test_generator)

#the model was bouncing between 50-55 % accuracy so i stopped the iterations and tried transfer learning 

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
109/109 [==============================] - 21s 191ms/step - loss: 1.0789 - accuracy: 0.4839 - val_loss: 1.1426 - val_accuracy: 0.3935
Epoch 2/100
109/109 [==============================] - 20s 188ms/step - loss: 1.0383 - accuracy: 0.4871 - val_loss: 1.4172 - val_accuracy: 0.4825
Epoch 3/100
109/109 [==============================] - 20s 185ms/step - loss: 0.9870 - accuracy: 0.4911 - val_loss: 4.5219 - val_accuracy: 0.3089
Epoch 4/100
109/109 [==============================] - 20s 187ms/step - loss: 0.9733 - accuracy: 0.4923 - val_loss: 4.3409 - val_accuracy: 0.4672
Epoch 5/100
 84/109 [======================>.......] - ETA: 2s - loss: 0.9874 - accuracy: 0.5058

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-ea5cab409e28>", line 1, in <module>
    r=model.fit_generator(train_generator,epochs=100,steps_per_epoch=10941//100,validation_data=test_generator)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 1829, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 1098, in fit
    tmp_logs = train_function(iterator)
  File "/usr/local/lib/pyt

KeyboardInterrupt: ignored

In [4]:
## transfer learning with augmentation
from tensorflow.keras.models import Model

In [15]:
pretrainedmodel=tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=[100,100,3]
)

83689472/83683744 [==============================] - 0s 0us/step


In [16]:
pretrainedmodel.trainable=False

In [17]:
x=Flatten()(pretrainedmodel.output)
x=Dense(512,activation='relu')(x)
x=Dense(8,activation='softmax')(x)


In [18]:
model=Model(inputs=pretrainedmodel.input,outputs=x)

In [19]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 49, 49, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 49, 49, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 49, 49, 32)   0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [20]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [21]:
r=model.fit_generator(train_generator,steps_per_epoch=10941/100,epochs=10,validation_data=test_generator)

Epoch 1/10
110/109 [==============================] - 24s 219ms/step - loss: 1.5736 - accuracy: 0.6673 - val_loss: 1.4466 - val_accuracy: 0.5777
Epoch 2/10
110/109 [==============================] - 23s 209ms/step - loss: 0.7709 - accuracy: 0.7270 - val_loss: 1.2063 - val_accuracy: 0.6090
Epoch 3/10
110/109 [==============================] - 23s 209ms/step - loss: 0.7304 - accuracy: 0.7409 - val_loss: 1.0864 - val_accuracy: 0.6116
Epoch 4/10
110/109 [==============================] - 23s 211ms/step - loss: 0.7078 - accuracy: 0.7457 - val_loss: 1.0732 - val_accuracy: 0.6313
Epoch 5/10
110/109 [==============================] - 23s 210ms/step - loss: 0.6808 - accuracy: 0.7543 - val_loss: 1.4441 - val_accuracy: 0.6105
Epoch 6/10
110/109 [==============================] - 23s 210ms/step - loss: 0.6455 - accuracy: 0.7580 - val_loss: 1.2256 - val_accuracy: 0.6327
Epoch 7/10
110/109 [==============================] - 23s 205ms/step - loss: 0.6683 - accuracy: 0.7645 - val_loss: 1.3660 - val_ac

In [22]:
model.save('emotion_recognition.h5')

In [23]:
from google.colab import files
files.download('emotion_recognition.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>